In [ ]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

## 📂 RAG 데이터 불러오기

이전에 `rag_data.ipynb`에서 저장한 Pickle 파일을 불러옵니다.

- **`pickle.load()`**: 저장된 파이썬 객체를 메모리로 다시 불러오기
- **`'rb'` 모드**: Read Binary (바이너리 읽기 모드)
- **`rag_data`**: 질문, 컨텍스트, 정답 등이 담긴 딕셔너리
- **목적**: RAG 시스템의 효과를 실험하기 위한 데이터 준비

In [ ]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

## 📊 테스트 데이터 준비

실험을 위해 처음 10개의 데이터만 추출합니다.

- **`questions`**: 10개의 질문 리스트
- **`contexts`**: 각 질문마다 3개씩의 컨텍스트 (총 10x3 구조)
- **`answers`**: 각 질문에 대한 정답 10개
- **`[:10]`**: 파이썬 슬라이싱으로 처음 10개만 선택
- **`len(questions)`**: 추출한 질문 개수 확인 (결과: 10)

### 목적
빠른 실험을 위해 전체 200개가 아닌 일부만 사용합니다.

In [ ]:
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

## 🤖 RAG 없이 GPT-4로 직접 답변 생성

**컨텍스트를 제공하지 않고** GPT-4에게 질문만 던져서 답변을 받습니다.

### 동작 과정
1. **반복문**: 10개 질문을 하나씩 처리
2. **프롬프트 작성**: 
   - "당신은 금융 전문가입니다"
   - 질문만 제공 (참고할 문서 없음)
3. **`call_openai()`**: GPT-4 모델에게 답변 요청
4. **결과 저장**: 생성된 답변을 `predictions` 리스트에 추가

### 핵심 포인트 ⚠️
- **컨텍스트 없음**: AI가 자신의 학습 데이터만으로 답변
- **한계**: 최신 정보나 특정 문서 내용은 모를 수 있음
- **비교 기준**: 나중에 RAG를 적용한 결과와 비교할 기준점

### 결과
10개 질문에 대한 GPT-4의 직접 답변이 `predictions`에 저장됩니다.

In [ ]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import answer_correctness


data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

## 📉 RAG 없는 답변의 성능 평가

컨텍스트 없이 생성한 답변의 정확도를 평가합니다.

### 평가 데이터 구성
- **`question`**: 사용자 질문 10개
- **`answer`**: GPT-4가 생성한 답변 (컨텍스트 없음)
- **`ground_truth`**: 실제 정답

### 평가 메트릭
- **`answer_correctness`**: 답변이 정답과 얼마나 일치하는지 측정
  - 0.0 ~ 1.0 사이 값
  - 1.0에 가까울수록 정확한 답변
  - **의미적 유사도 + 사실 정확도** 모두 고려

### 이 점수의 의미
- **기준점(Baseline)**: RAG를 적용하지 않은 상태의 성능
- **비교 대상**: 다음 단계에서 RAG를 적용한 후의 성능과 비교
- **예상 결과**: RAG를 적용하면 이 점수보다 높아질 것으로 기대

In [ ]:
print(questions[0])

## 🔍 첫 번째 질문 확인

테스트 데이터의 첫 번째 질문을 출력합니다.

- **목적**: 어떤 질문에 대해 평가하고 있는지 확인
- **예시**: "글로벌 저금리 현상이 부각된 원인은 무엇인가요?"
- **다음 셀과 연결**: 이 질문에 대한 GPT-4의 답변을 확인할 예정

In [ ]:
print(predictions[0])

## 💬 RAG 없는 답변 확인

컨텍스트 없이 GPT-4가 생성한 첫 번째 답변을 출력합니다.

### 특징
- **컨텍스트 없음**: 특정 문서를 참고하지 않고 생성
- **일반적인 답변**: GPT-4의 학습 데이터에 기반한 일반적 지식
- **한계점**: 
  - 구체적인 세부사항이 부족할 수 있음
  - 최신 정보가 반영되지 않을 수 있음
  - 특정 문서의 정확한 내용과 다를 수 있음

### 비교 준비
다음 단계에서 **RAG를 적용한 답변**과 비교하여 얼마나 개선되는지 확인할 예정입니다.

In [ ]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity


def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

## 🎯 RAG 시스템 구현 및 답변 생성

**컨텍스트를 제공하여** GPT-4가 더 정확한 답변을 생성하도록 합니다.

### `retrieve_context()` 함수 - RAG의 핵심!
```python
def retrieve_context(question, contexts):
    # 1단계: 질문을 임베딩으로 변환
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    
    # 2단계: 모든 컨텍스트를 임베딩으로 변환
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')
    
    # 3단계: 질문과 각 컨텍스트 간 유사도 계산
    similarities = [cosine_similarity(question_embedding, ce) for ce in context_embeddings]
    
    # 4단계: 가장 유사도가 높은 컨텍스트 찾기
    most_relevant_index = np.argmax(similarities)
    
    # 5단계: 가장 관련 있는 컨텍스트 반환
    return contexts[most_relevant_index]
```

### 전체 RAG 프로세스
1. **검색(Retrieve)**: 질문과 가장 관련 있는 컨텍스트 찾기
2. **증강(Augment)**: 찾은 컨텍스트를 프롬프트에 추가
3. **생성(Generate)**: 컨텍스트를 참고하여 GPT-4가 답변 생성

### 프롬프트 차이
- **이전**: "질문에 답하세요"
- **현재**: "Context를 적절히 활용하여 질문에 답하세요"

### 기대 효과 ✨
- 특정 문서의 정확한 정보 활용
- 구체적이고 상세한 답변
- 사실 기반의 정확한 답변

In [ ]:
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_relevancy, context_recall, context_precision


data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

## 📈 RAG 적용 후 성능 평가

컨텍스트를 제공한 후의 답변 정확도를 평가합니다.

### 평가 데이터 구성
- **`question`**: 동일한 10개 질문
- **`answer`**: RAG를 적용하여 생성한 답변 (컨텍스트 있음)
- **`ground_truth`**: 동일한 정답

### 평가 메트릭
- **`answer_correctness`**: 답변의 정확도 측정

### 비교 분석 🔍
| 항목 | RAG 없음 | RAG 적용 |
|------|---------|---------|
| 컨텍스트 | ❌ 없음 | ✅ 있음 |
| 정보 출처 | 학습 데이터 | 특정 문서 |
| 정확도 | 낮음 | **높음** |

### 기대 결과
- RAG 없는 경우보다 **점수가 높아질 것으로 예상**
- 컨텍스트 제공으로 인한 답변 품질 향상 확인
- **RAG의 효과를 정량적으로 증명**

In [ ]:
print(predictions[0])

## ✨ RAG 적용 답변 확인

컨텍스트를 활용하여 생성한 첫 번째 답변을 출력합니다.

### 특징
- **컨텍스트 있음**: 관련 문서를 참고하여 생성
- **구체적인 답변**: 특정 문서의 정보 기반
- **개선점**:
  - 더 구체적이고 상세한 내용
  - 문서에 기반한 정확한 정보
  - 사실 관계가 명확함

### 비교해볼 점 🔍
이전 셀(RAG 없는 답변)과 비교하여:
- 답변의 구체성이 향상되었는가?
- 특정 문서의 정보가 반영되었는가?
- 정확도가 개선되었는가?

In [ ]:
print(answers[0])

## ✅ 실제 정답 확인

첫 번째 질문의 실제 정답(ground truth)을 출력합니다.

### 목적
- **최종 비교**: RAG 없는 답변 vs RAG 적용 답변 vs 실제 정답
- **평가 기준**: RAGAS가 이 정답을 기준으로 점수를 매김
- **품질 확인**: 생성된 답변이 실제 정답과 얼마나 유사한지 육안으로 확인

### 3가지 답변 비교 요약

| 구분 | 특징 | 정확도 |
|------|------|--------|
| **RAG 없음** | 일반적 지식 | 낮음 |
| **RAG 적용** | 문서 기반 | **높음** ⬆️ |
| **실제 정답** | 기준값 | 100% |

### 핵심 인사이트 💡
RAG를 적용하면 실제 정답에 더 가까운 답변을 생성할 수 있습니다!